In [10]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.model_selection import train_test_split

# from surprise import Dataset
# from surprise import Reader
# from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
# from surprise import SVD, KNNBasic, accuracy

# **Content-based model**

Подход **content-based** предполагает, что пользователю рекомендуются товары или контент на основе его предпочтений и вкусов.

### КОСИНУСНАЯ БЛИЗОСТЬ

In [11]:
a = np.array([1.1, 2.3, 5.1])
b = np.array([1.3, 2.1, 4.9])
c = np.array([5.1, 6.2, 1.1])

**Задание 2.1**

Вычислите косинусную близость между векторами А и С. Результат округлите до трёх знаков после точки-разделителя.

In [12]:
(np.dot(a, c)/(np.linalg.norm(a)*np.linalg.norm(c))).round(3)

0.551

## Построение рекомендательной системы на основе контента

Подход **TF-IDF** (Term Frequency-Inverse Document Frequency) - преобразование текста в вектор

Показатель **TD-IDF** — это индикатор того, насколько релевантно слово в контексте документа.

In [13]:
df = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/747dae7bf99b18ce3b24bd34aa7bc29b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/netflix_titles.zip')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [14]:
# стоп-слова - предлоги и другие служебные части речи, которые не несут содержательной информации
model = TfidfVectorizer(stop_words='english')

# Заполним пропуски пустыми строками:
df['description'] = df['description'].fillna('')

# Трансформируем наши описания в матрицу
feature_matrix = model.fit_transform(df['description'])

**Задание 2.2**

Сколько столбцов в получившейся матрице?

In [15]:
feature_matrix.shape[1]

17905

In [34]:
# Вычисляем косинусную близость

cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [17]:
# Вернём индексацию и уберём дубликаты из данных:

indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [18]:
# Функция для создания рекомендаций

def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

**Задание 2.3**

Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году

In [19]:
get_recommendations('Balto').values[1]

'Vroomiz'

# **КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ**

In [ ]:
# Создаём объект класса Dataset для работы с модулем surprise
# datas = Dataset.load_from_file(
#     "u.data.txt",
#     reader=Reader(line_format="user item rating timestamp", sep="\t"),
# )
dataa = Dataset.load_builtin('ml-100k')

In [21]:
data = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/6e47046882bad158b0efbb84cd5cb987/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/u.data.txt',
                   sep="\t",
                   header=None,
                   names=['userId', 'movieId', 'rating', 'timestamp'])
data

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


**Задание 3.1**

Сколько уникальных фильмов в наборе данных?

In [22]:
data.movieId.nunique()

1682

**Задание 3.2**

Сколько уникальных пользователей в наборе данных?

In [23]:
len(set(data.userId.values))

943

**Задание 3.3**

Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.

In [24]:
data.rating.value_counts().index[:1]

Index([4], dtype='int64', name='rating')

**Задание 3.4**

Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. В качестве значения параметра random_state возьмите число 13.

Сколько объектов попало в тестовую выборку?

In [ ]:
trainset, testset = train_test_split(dataa, test_size=0.25, random_state=13)
len(testset)

In [ ]:
# Предсказываем с помощью KNNBasic
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options).fit(trainset)
pred_memoryb = knn.test(testset)

**Задание 3.5**

1. Каков реальный рейтинг, выставленный пользователем с ID 500 для фильма с ID 699?
2. Каков прогнозируемый рейтинг для пользователя с ID 500 и фильма с ID 699? Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
for y in pred_memoryb: 
    if y.uid == '500' and y.iid == '699':
        print('1 - ', y.r_ui, '2 - ', y.est.round(2))

In [ ]:
# Оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу

pred = pd.DataFrame(pred_memoryb)
pred.sort_values(by=['est'], inplace=True, ascending = False)

# Пример рекомендаций для конкретного пользователя

pred[pred.uid =='849']['iid'].to_list()

**Задание 3.6**

Реализуйте user-based-алгоритм. Какое значение RMSE получилось для коллаборативной фильтрации типа user-based? Ответ округлите до двух знаков после точки-разделителя.

accuracy.rmse(pred_memoryb).round(2)

**Задание 3.7**

Теперь давайте сравним полученные результаты с результатами SVD-алгоритма. Реализуйте SVD с параметрами по умолчанию.

Какое значение RMSE получилось для SVD? Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
svd = SVD().fit(trainset)
pred_modelb = svd.test(testset)
accuracy.rmse(pred_modelb).round(2)

# **ГИБРИДНЫЕ МОДЕЛИ**

In [26]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

from scipy.sparse import csr_matrix

In [27]:
path = 'data/Gooddreadbooks/'
ratings = pd.read_csv(path + 'ratings.csv') # Поставленные оценки
books = pd.read_csv(path + 'books.csv') # Информация о книгах
tags = pd.read_csv(path + 'tags.csv') # Информация о тегах
book_tags = pd.read_csv(path + 'book_tags.csv') # Книги с тегами 

**Задание 4.1**

Добавьте в набор данных book_tags признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

Какой обычный id у книги, которая имеет id 5 в системе Goodreads?

In [28]:
bid_gbid = dict(zip(books.goodreads_book_id,books.book_id))

book_tags['book_id'] = book_tags.goodreads_book_id.map(bid_gbid)
book_tags[book_tags.goodreads_book_id == 5].book_id.values[0]

18

**Задание 4.2**

Далее нам необходимо оставить в наборе данных book_tags только те записи, теги для которых есть в данных tags.

Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, в которых находятся теги, информация о которых есть в наборе данных tags.

Сколько объектов осталось?

In [29]:
book_tags_1 = book_tags.merge(tags, how='right')
book_tags_1.shape[0]

300738

In [30]:
# выставленный рейтинг - значения матрицы, id пользователя и id книги - индексы для строк и столбцов матрицы

ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id, ratings.book_id)))

In [31]:
# матрицу с метаданными (id книги на id тега) - если у книги есть тег, то значение 1

meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.book_id,book_tags.tag_id))) 

**Задание 4.4**

Давайте проверим, что всё получилось правильно.

Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

In [32]:
ratings_matrix.mean().round(3)

0.007

In [ ]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

In [ ]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

**Задание 4.5**

Оцените качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных (item_features = meta_matrix).

In [ ]:
prec_score = precision_at_k(
                     model,
                     test,
                     item_features = meta_matrix).mean() 
print(prec_score)

In [ ]:
# Предсказание рейтинга для нового пользователя

scores = model.predict(655, np.arange(n_items), user_features=new_user_feature)

# **Глубокое обучение (Deep Learning, DL)**

In [35]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [37]:
df = pd.read_csv(path + 'ratings.csv')
df

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


**Задание 5.1**

Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random_state возьмите число 42.

Сколько объектов теперь находится в обучающей выборке?

In [38]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape[0])

785404


**Задание 5.2**

Запишите количество уникальных книг в переменную n_books.

Сколько в наборе данных уникальных книг?

In [39]:
n_books = df.book_id.nunique()
n_books

10000

**Задание 5.3**

Запишите количество уникальных пользователей в переменную n_users.

Сколько в наборе данных уникальных пользователей?

In [40]:
n_users = df.user_id.nunique()
n_users

53424

In [42]:
# Создаём эмбеддинги для книг

book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [43]:
# Создаём эмбеддинги для пользователей:

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [44]:
# Соединяем эмбендинги
conc = Concatenate()([book_vec, user_vec])

In [46]:
# Собираем нейронную сеть из трех слоёв (128, 32 и 1 нейрон)
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

model2 = Model([user_input, book_input], out)

In [47]:
# Задаём алгоритм оптимизации и метрику, которую мы будем оптимизировать

model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [48]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 94s 4ms/step - loss: 0.8008
Epoch 2/5
24544/24544 [==============================] - 93s 4ms/step - loss: 0.6882
Epoch 3/5
24544/24544 [==============================] - 86s 3ms/step - loss: 0.6578
Epoch 4/5
24544/24544 [==============================] - 81s 3ms/step - loss: 0.6328
Epoch 5/5
24544/24544 [==============================] - 75s 3ms/step - loss: 0.6093


In [49]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 7s 1ms/step - loss: 0.7094


0.7094364762306213

In [50]:
# Настройка модели

fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 81s 3ms/step - loss: 0.6278
Epoch 2/10
24544/24544 [==============================] - 92s 4ms/step - loss: 0.5746
Epoch 3/10
24544/24544 [==============================] - 88s 4ms/step - loss: 0.5542
Epoch 4/10
24544/24544 [==============================] - 102s 4ms/step - loss: 0.5375
Epoch 5/10
24544/24544 [==============================] - 89s 4ms/step - loss: 0.5240
Epoch 6/10
24544/24544 [==============================] - 76s 3ms/step - loss: 0.5125
Epoch 7/10
24544/24544 [==============================] - 85s 3ms/step - loss: 0.5025
Epoch 8/10
24544/24544 [==============================] - 88s 4ms/step - loss: 0.4940
Epoch 9/10
24544/24544 [==============================] - 93s 4ms/step - loss: 0.4866
Epoch 10/10
6136/6136 [==============================] - 7s 1ms/step - loss: 0.7832


0.7831881642341614